## pg_restore cipsrv_nhdplus_h

In [ ]:
import os,sys,psycopg2;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

s3bk = os.environ['S3_BUCKET'];
# override: s3bk = 'https://edap-ow-data-commons.s3.amazonaws.com/data/';
lddk = os.path.join(os.path.expanduser('~'),'loading_dock');
dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
thds = 2;

src = 'cipsrv_nhdplus_h_2.dmp';
dmp = 'cipsrv_nhdplus_h.dmp';

print("source: " + s3bk);

In [ ]:
z = common.download(
     s3  = s3bk
    ,src = src
    ,ld  = lddk
    ,trg = dmp
);

if not os.path.exists(os.path.join(lddk,dmp)):
    raise Exception('failed to download dmp file');

print("successfully downloaded " + src);    

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
with closing(conn.cursor()) as cursor:
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdplusflowlinevaa_catnodes CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdplusflowlinevaa_levelpathi CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_3338    CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_5070    CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_26904   CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_32161   CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_32655   CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.catchment_32702   CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_3338  CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_5070  CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_26904 CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_32161 CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_32655 CASCADE");
    cursor.execute("DROP MATERIALIZED VIEW IF EXISTS cipsrv_nhdplus_h.nhdflowline_32702 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_nhdplus_h.catchment_fabric CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_nhdplus_h.nhdflowline CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_nhdplus_h.nhdplusflowlinevaa CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_nhdplus_h.nhdplusflow CASCADE");
    conn.commit();

print("preexisting data purged.");    

In [ ]:
z = common.pg_restore(
     host
    ,port
    ,dbse
    ,user
    ,pswd
    ,os.path.join(lddk,dmp)
    ,thds
);

print("pg_restore complete.");

In [ ]:
if z[0] != b'':
    print(str(z));
    sys.exit(-10);
